- Autor: Virginia Monroy Malca
- Correo: viriginia.monroy@ibm.com

# Despliegue modelo

## Definiendo librerías

In [1]:
#1
import pandas as pd
import numpy as np
# Librería de XGBoost
import xgboost as xgb
# Librería para instanciar Watson Machine Learning
from watson_machine_learning_client import WatsonMachineLearningAPIClient

## Credenciales de WML

In [2]:
#2
# Instanciar las credenciales de wml aquí
wml_credentials = {"Insertar las credenciales aquí"}

client = WatsonMachineLearningAPIClient(wml_credentials)

## Cargar el modelo de WML Respositorio


In [ ]:
#3
client.repository.list_models()

In [4]:
#4
published_model_uid = '*******************************'

In [5]:
#5
loaded_model = client.repository.load(published_model_uid)

In [ ]:
#6
type(loaded_model)

In [7]:
#7
f_names = loaded_model.feature_names

## Importando las credenciales de ICOS

In [3]:
#8
"Insertar credenciales ICOS"

## Importando la data

In [ ]:
#9
# Descargando 
cos.download_file(Bucket=credentials['BUCKET'],Key='telco_new.csv',Filename='telco_new.csv')
data = pd.read_csv('telco_new.csv', sep = ";")
data.head()

In [ ]:
#10
data.shape

## Missings

In [6]:
#11
data['Total_Charges'] = pd.to_numeric(data['Total_Charges'], errors='coerce')

In [7]:
#12
# Asignando missings
# Total_Charges -> promedio
data.Total_Charges.fillna(np.mean(data.Total_Charges),inplace=True)

## Dummies

In [8]:
#13
data = pd.get_dummies(data)

## PCA

In [ ]:
#14
# Descargando librería
from sklearn.decomposition import PCA
# Generando objeto
pca=PCA(n_components=33)
# Entrenando y aplicando al dataset
data_pca=pca.fit_transform(data.iloc[:,3:])
df_pca = pd.DataFrame(data_pca)
data_final = pd.concat([data, df_pca], axis=1)
data = data_final
data.head()

## Separando X de Y

In [ ]:
#15
##Para cambiar el tipo de variables
f_names_rf = []
for i in f_names:
    if i.isdigit():
        f_names_rf.append(int(i))
    else:
        f_names_rf.append(i)

In [ ]:
#16
##Define X and Y
X=data[f_names_rf]
print("DONE")

In [ ]:
#17
X_to_pred = xgb.DMatrix(X)

## Prediciendo los valores de X

In [ ]:
#18
X_to_pred

In [20]:
#19
pred = loaded_model.predict(X_to_pred)
pred_val = []
for i in pred:
    if i>=0.5:
        pred_val.append(1)
    else:
        pred_val.append(0)

## Almacenando la predicción en un DataFrame

In [ ]:
#20
data['pred'] = pred_val
data.head()

In [23]:
#21
data.to_csv("resultados_telco_new_vmm.csv")

## Guardando el scoring en ICOS

In [24]:
#22
# Guardando el scoring
cos.upload_file(Filename='resultados_telco_new_vmm.csv',Bucket='model-resultados',Key='resultados_telco_new_vmm.csv')